## Example 01 - Building a Layered Dike

This example defines a digital twin simulation in which several layers (from different materials) of a dike are constructed.

It starts with importing the required libraries, that is done by the next cell. After that, all example-specific relations and objects are defined.

[**Work method:**](#Define-work-method)<br>
Move sand from the *from_site* to the *to_site*.<br>
Use one trailing suction hopper dredger to move the material.<br>
Project is finished if the *from_site* is depleted and the *to_site* is full.

[**1. Sites:**](#1.-Define-the-project-sites)<br>
One dredging site.<br>
One placement site.

[**2. Equipment:**](#2.-Define-the-project-equipment)<br>
One trailing suction hopper dredger.

*Hopper can only dredge with certain tidal / wave conditions!*

[**3. Activities:**](#3.-Define-the-activity)<br>
Define the work method into an activity object.

[**4. Run the simulation:**](#4.-Run-the-simulation)<br>
Define the work method into an activity object.

[**5. Post processing:**](#5.-Postprocessing)<br>
Gantt chart

In [36]:
# package(s) related to time, space and id
import datetime, time
import platform

# you need these dependencies (you can get these from anaconda)
# package(s) related to the simulation
import simpy

# spatial libraries 
import shapely.geometry
from simplekml import Kml, Style

# package(s) for data handling
import numpy as np

# digital twin package
import digital_twin.core as core
import digital_twin.model as model
import digital_twin.plot as plot

In [37]:
# Create simulation environment
simulation_start = datetime.datetime.now()

my_env = simpy.Environment(initial_time = time.mktime(simulation_start.timetuple()))
my_env.epoch = time.mktime(simulation_start.timetuple())

## Define work method

### 1. Define the project sites
You can specify a project site object by entering mix-ins:

    core.Identifiable - enables you to give the object a name
    core.Log - enables you to log all discrete events in which the object is involved
    core.Locatable - enables you to add coordinates to extract distance information and visualize
    core.HasContainer - enables you to add information on the material available at the site
    core.HasResource - enables you to add information on serving equipment
    core.HasWeather - enables you to add weather conditions

#### First create objects with the desired properties

In [38]:
Site = type('Site', (core.Identifiable, core.Log, core.Locatable, core.HasContainer, core.HasResource), {})

#### Next specify the properties for sites you wish to create

In [39]:
# sites data in database
data_stock_clay = {"env": my_env,
                   "name": "Clay_Stock",
                   "geometry": shapely.geometry.Point(5.019298185633251, 52.94239823421129),  # lon, lat
                   "capacity": 150000,
                   "level": 150000}

data_stock_sand = {"env": my_env,
                   "name": "Sand_Stock",
                   "geometry": shapely.geometry.Point(5.271417603333632, 52.9638452897506),  # lon, lat
                   "capacity": 150_000,
                   "level": 150_000}

data_stock_armour = {"env": my_env,
                   "name": "Armour_Stock",
                   "geometry": shapely.geometry.Point(5.919298185633251, 52.94239823421129),  # lon, lat
                   "capacity": 150_000,
                   "level": 150_000}

data_stock_levvel = {"env": my_env,
                   "name": "Levvel_Stock",
                   "geometry": shapely.geometry.Point(5.919298185633251, 52.94239823421129),  # lon, lat
                   "capacity": 150_000,
                   "level": 150_000}

In [40]:
site_count = 20

# layer data from database
data_clay = {"start": [5.054676856441372, 52.94042293840172], # Den Oever (lon, lat)
             "stop": [5.294877712236641, 53.06686424241725], # Kornwerderzand (lon, lat)
             "capacity": 5_000,
             "name": "Clay_Layer"}
data_sand = {"start": [5.052051052879287,52.9421894472733], # Den Oever (lon, lat)
             "stop": [5.292216781509101,53.06886359869087], # Kornwerderzand (lon, lat)
             "capacity": 5_000,
             "name": "Sand_Layer"}
data_armour = {"start": [5.049510554598302,52.94393628899332], # Den Oever (lon, lat)
             "stop": [5.289636346490858,53.07053144816584], # Kornwerderzand (lon, lat)
             "capacity": 5_000,
             "name": "Armour_Layer"}
data_levvel = {"start": [5.046556507026805,52.94579445406793], # Den Oever (lon, lat)
             "stop": [5.286775240694118,53.07264015015531], # Kornwerderzand (lon, lat)
             "capacity": 5_000,
             "name": "Levvel_Layer"}

#### Finally create specific instances of the predefined objects with the specified properties

In [41]:
# create the site objects
stocks = []
for data in [data_stock_clay, data_stock_sand, data_stock_armour, data_stock_levvel]:
    stock = Site(**data)
    stocks.append(stock)

In [42]:
sites = []
for data_layer in [data_clay, data_sand, data_armour, data_levvel]:
    start = data_layer['start']
    stop = data_layer['stop']

    # generate the evenly spaced locations between the selected start and stop points
    lons = np.linspace(start[0], stop[0], num=site_count)
    lats = np.linspace(start[1], stop[1], num=site_count)

    # create a Site object for each location
    layer_sites = []
    for i in range(site_count):
        data_site = {"env": my_env,
                     "name": data_layer["name"] + '_' + format(i, '02.0f'),
                     "geometry": shapely.geometry.Point(lons[i], lats[i]),
                     "capacity": data_layer["capacity"]}
        site = Site(**data_site)
        layer_sites.append(site)
    sites.append(layer_sites)

### 2. Define the project equipment
You can specify a vessel object by entering mix-ins:

    core.Identifiable - enables you to give the object a name
    core.Log - enables you to log all discrete events in which the object is involved
    core.ContainerDependentMovable - A moving container, so capacity and location
    core.Processor - Allow for loading and unloading
    core.HasResource - Add information on serving equipment
    core.HasDepthRestriction - Add information on depth restriction 

For this case, we want to define a trailing suction hopper dredger. Which basically is a moving container, hence the core.ContainerDependentMovable. Additionally it is a processor (dredging and dumping), core.Processor. It also requires the capacity to serve barges for example, core.HasResource. 

#### First create objects with the desired properties

In [43]:
TransportResource = type('TransportResource', 
                         (core.Identifiable, core.Log, core.ContainerDependentMovable, 
                          core.HasResource), {})

TransportProcessingResource = type('TransportProcessingResource', 
                                   (core.Identifiable, core.Log, core.ContainerDependentMovable, 
                                    core.Processor, core.HasResource), {})

ProcessingResource = type('ProcessingResource', 
                          (core.Identifiable, core.Log, core.Processor, core.HasResource, core.Locatable), {})

#### Next specify the properties for vessel(s) you wish to create

In [44]:
# For more realistic simulation you might want to have speed dependent on the volume carried by the vessel
def compute_v_provider(v_empty, v_full):
    return lambda x: x * (v_full - v_empty) + v_empty

def compute_loading(rate):
    return lambda x: x / rate

def compute_unloading(rate):
    return lambda x: x / rate

In [45]:
# equipment information in database
data_gantry_crane = {"env": my_env,
                     "name": "Gantry crane",
                     "geometry": shapely.geometry.Point(5.019298185633251, 52.94239823421129),  # lon, lat
                     "loading_func": compute_loading(0.1),                # Loading rate
                     "unloading_func": compute_unloading(0.1)}            # Unloading rate


data_installation_crane = {"env": my_env,
                           "name": "Installation crane",
                           "geometry": shapely.geometry.Point(5.019298185633251, 52.94239823421129),  # lon, lat
                           "loading_func": compute_loading(0.5),                # Loading rate
                           "unloading_func": compute_unloading(0.5)}            # Unloading rate


data_transport_barge_01 = {"env": my_env,
                           "name": "Transport barge 01",
                           "geometry": shapely.geometry.Point(5.019298185633251, 52.94239823421129),  # lon, lat
                           "capacity": 1_000,
                           "compute_v": compute_v_provider(v_empty=1.6, v_full=1)}

data_transport_barge_02 = {"env": my_env,
                           "name": "Transport barge 02",
                           "geometry": shapely.geometry.Point(5.019298185633251, 52.94239823421129),  # lon, lat
                           "capacity": 1_000,
                           "compute_v": compute_v_provider(v_empty=1.6, v_full=1)}

# TSHD variables
data_hopper = {"env": my_env,                                       # The simpy environment 
               "name": "Hopper",                                    # Name
               "geometry": shapely.geometry.Point(5.070195628786471, 52.93917167503315),                      # It starts at the "from site"
               "loading_func": compute_loading(1.5),                # Loading rate
               "unloading_func": compute_unloading(1.5),            # Unloading rate
               "capacity": 1_000,                                   # Capacity of the hopper - "Beunvolume"
               "compute_v": compute_v_provider(2, 1.5)}             # Variable speed 

#### Finally create specific instances of the predefined objects with the specified properties

In [46]:
# create the processing resources
gantry_crane = ProcessingResource(**data_gantry_crane)
installation_crane = ProcessingResource(**data_installation_crane)

# create the transport resources
transport_barge_01 = TransportResource(**data_transport_barge_01)
transport_barge_02 = TransportResource(**data_transport_barge_02)

# create the transport processing resource
hopper = TransportProcessingResource(**data_hopper)

### 3. Define the activity

In [47]:
activities = []

clay_layer_index = 0
clay_sites = sites[clay_layer_index]
for i, clay_site in enumerate(clay_sites):
    for barge_index, transport_barge in enumerate([transport_barge_01, transport_barge_02]):
        # for the first site we do not need to define any condition
        condition = None if i == 0 else model.LevelCondition(container=clay_sites[i-1], min_level=data_clay["capacity"])
        activity = model.Activity(env=my_env,
                                  name="Clay_Placement_S" + format(i+1, '02.0f') + "_B" + format(barge_index + 1, '02.0f'),
                                  origin=stocks[clay_layer_index],
                                  destination=clay_site,
                                  loader=gantry_crane,
                                  mover=transport_barge,
                                  unloader=installation_crane,
                                  start_condition=condition)
        activities.append(activity)

As can be seen in the code fragment above, the Activity class takes the Simpy environment and a name as its constructor arguments. This is because it is defined as a mixin of the SimpyObject and Identifiable core classes. To construct an activity we also needed to define the origin, destination, loader, mover and unloader. Finally, we can optionally define conditions that need to be satisfied for the transportation of materials to take place. These are separated in __start_condition__, __stop_condition__ and __condition__ arguments:
* __start_condition__: The activity will start as soon as this condition is satisfied. If this condition is not yet satisfied at the start of the simulation the activity will be suspended untill the condition is satisfied at a later time. Since we do not want any clay to be transported to the later sites if the previous site has not yet been completed, we define this condition as a start_condition.
* __stop_condition__: After the activity has started (the start_condition was satisfied), it will regularly check if the stop_condition has been satisfied. If so, the activity is considered complete and will terminate. A simulation will not terminate untill all of its activities have been completed. By default (with stop_condition=None), the stop_condition is set to either the origin container being empty, or the destination container being full. This is sufficient in our case, so we do not specify a stop_condition. 
* __condition__: After the activity has started (the start_condition was satisfied) and as long as the stop_condition is not satisfied, the activity will continuously check whether this condition is satisfied. If so, it will complete exactly one transportation of materials (loading the mover at the origin and unloading the mover at the destination). If not, the activity will wait untill the condition is satisfied again. Defining such a condition allows for the activity to be temporarily halted, for example due to bad weather or other working conditions, and resuming at a later time when the problem preventing work has passed. 

All of these condition arguments take a "Condition" object. This object should have a "satisfied" method which returns a boolean indicating whether the condition is satsified or not. The digital twin package provides a LevelCondition, allowing you to quickly construct a condition which will check if an object extending HasContainer has a certain minimum and/or maximum level. It also provides an AndCondition and OrCondition which allow for combining other condition objects into a single condition. You can also provide your own "Condition" class, for example you could program a "Condition" class which depending on the time in the simulation (the .now fixture of the Simpy environment), returns whether the weather is good enough for work to take place:

In [48]:
class WeatherCondition:
    def __init__(self, env, *args, **kwargs):
        super().__init__(*args, **kwargs)
        """Initialization"""
        self.env = env
    
    def satisfied(self):
        return self.env.now < 30_000 or self.env.now > 40_000  # between time step 30_000 and 40_000 a "storm" takes place

For our simple dike building example we do not define any advanced conditions. We do need to define Activities for the other layers (sand, armour and levvel). For each of these layers we again require that they are installed in order. We also require the previous layer to be completed, before the next layer can be installed. To achieve this we combine two LevelConditions with an AndCondition. We choose to install the sand layer using the hopper, and the armour and levvel layers using the transport barges and grantry and installation cranes.

In [49]:
# define the sand layer Activities
sand_layer_index = 1
sand_sites = sites[sand_layer_index]
for i, sand_site in enumerate(sand_sites):
    # for the first site we do not need to define a condition on the previous site
    prev_site_condition = model.TrueCondition() if i == 0 else model.LevelCondition(container=sand_sites[i-1], 
                                                                          min_level=sand_sites[i-1].container.capacity)
    prev_layer_sites = sites[clay_layer_index]
    prev_layer_condition = model.LevelCondition(prev_layer_sites[i], min_level=prev_layer_sites[i].container.capacity)
    start_condition = model.AndCondition([prev_site_condition, prev_layer_condition])
    activity = model.Activity(env=my_env,
                              name="Sand_Placement_S" + format(i+1, '02.0f'),
                              origin=stocks[sand_layer_index],
                              destination=sand_site,
                              loader=hopper,
                              mover=hopper,
                              unloader=hopper,
                              start_condition=start_condition)
    activities.append(activity)

# define the armour and level layer activities
layer_names = ["Armour_Placement", "Levvel_Placement"]
layer_indices = [2, 3]
for layer_index in layer_indices:
    for barge_index, transport_barge in enumerate([transport_barge_01, transport_barge_02]):
        layer_sites = sites[layer_index]
        for site_index, site in enumerate(layer_sites):
            # for the first site we do not need to define a condition on the previous site
            prev_site_condition = model.TrueCondition() if site_index == 0 else \
                model.LevelCondition(container=layer_sites[site_index-1], 
                                     min_level=layer_sites[site_index-1].container.capacity)
            
            prev_layer_sites = sites[layer_index - 1]
            prev_layer_condition = model.LevelCondition(container=prev_layer_sites[site_index],
                                                        min_level=prev_layer_sites[site_index].container.capacity)
            start_condition = model.AndCondition([prev_site_condition, prev_layer_condition])
            
            activity = model.Activity(env=my_env,
                                      name=layer_names[layer_index-2] + "_S" + format(site_index+1, '02.0f')
                                           + "_B" + format(barge_index+1, '02.0f'),
                                      origin=stocks[layer_index],
                                      destination=site,
                                      loader=gantry_crane,
                                      mover=transport_barge,
                                      unloader=installation_crane,
                                      start_condition=start_condition)
            activities.append(activity)

### 4. Run the simulation

In [50]:
my_env.epoch = datetime.datetime.now()

my_env.run()

print("\n*** Installation of dike finished in {} ***".format(datetime.timedelta(seconds=int(my_env.now))))

T=1548952004.00 Start condition is satisfied, Clay_Placement_S01_B01 transporting from Clay_Stock to Clay_Layer_00 started
T=1548952004.00 Start condition is satisfied, Clay_Placement_S01_B02 transporting from Clay_Stock to Clay_Layer_00 started
T=1548952004.00 Clay_Placement_S02_B01 to Clay_Layer_01 suspended
T=1548952004.00 Clay_Placement_S02_B02 to Clay_Layer_01 suspended
T=1548952004.00 Clay_Placement_S03_B01 to Clay_Layer_02 suspended
T=1548952004.00 Clay_Placement_S03_B02 to Clay_Layer_02 suspended
T=1548952004.00 Clay_Placement_S04_B01 to Clay_Layer_03 suspended
T=1548952004.00 Clay_Placement_S04_B02 to Clay_Layer_03 suspended
T=1548952004.00 Clay_Placement_S05_B01 to Clay_Layer_04 suspended
T=1548952004.00 Clay_Placement_S05_B02 to Clay_Layer_04 suspended
T=1548952004.00 Clay_Placement_S06_B01 to Clay_Layer_05 suspended
T=1548952004.00 Clay_Placement_S06_B02 to Clay_Layer_05 suspended
T=1548952004.00 Clay_Placement_S07_B01 to Clay_Layer_06 suspended
T=1548952004.00 Clay_Placeme

Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
T=1549465285.28 Stop condition is satisfied, Armour_Plac

Nothing to move
Nothing to move
T=1551040004.00 Start condition is satisfied, Armour_Placement_S05_B01 transporting from Armour_Stock to Armour_Layer_04 started
Nothing to move
T=1551040004.00 Start condition is satisfied, Armour_Placement_S05_B02 transporting from Armour_Stock to Armour_Layer_04 started
T=1551040392.06 Stop condition is satisfied, Armour_Placement_S04_B02 transporting from Armour_Stock to Armour_Layer_03 complete
T=1551041904.58 Stop condition is satisfied, Armour_Placement_S04_B01 transporting from Armour_Stock to Armour_Layer_03 complete
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Noth

Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
T=1553434004.00 Start condition is satisfied, Armour_Placement_S09_B01 transporting from Armour_Stock to Armour_Layer_08

T=1555641813.86 Stop condition is satisfied, Sand_Placement_S16 transporting from Sand_Stock to Sand_Layer_15 complete
Nothing to move
Nothing to move
Nothing to move
T=1555680234.00 Stop condition is satisfied, Armour_Placement_S12_B01 transporting from Armour_Stock to Armour_Layer_11 complete
T=1555680404.00 Start condition is satisfied, Armour_Placement_S13_B01 transporting from Armour_Stock to Armour_Layer_12 started
T=1555680404.00 Start condition is satisfied, Armour_Placement_S13_B02 transporting from Armour_Stock to Armour_Layer_12 started
T=1555681034.00 Stop condition is satisfied, Armour_Placement_S12_B02 transporting from Armour_Stock to Armour_Layer_11 complete
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to mo

Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
T=1557235604.00 Start condition is satisfied, Levvel_Placement_S14_B01 transporting from Levvel_Stock to Levvel_Layer_13 started
Nothing to move
T=1557235604.00 Start condition is satisfied, Levvel_Placement_S14_B02 transporting from Levvel_Stock to Levvel_Layer_13 started
T=1557237011.32 Stop condition is satisfied, Levvel_Placement_S13_B02 transporting from Levvel_Stock to Levvel_Layer_12 complete
T=1557237063.59 Stop condition is satisfied, Levvel_Placement_S13_B01 transporting from Levvel_Stock to Levvel_Layer_12 complete
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
T=1557269821.73 Stop condition is satisfied, Sand_Placement_S19 transporting from Sa

Nothing to move
T=1559355110.03 Stop condition is satisfied, Armour_Placement_S19_B02 transporting from Armour_Stock to Armour_Layer_18 complete
T=1559356004.00 Start condition is satisfied, Armour_Placement_S20_B01 transporting from Armour_Stock to Armour_Layer_19 started
T=1559356004.00 Start condition is satisfied, Armour_Placement_S20_B02 transporting from Armour_Stock to Armour_Layer_19 started
Nothing to move
T=1559356605.01 Stop condition is satisfied, Armour_Placement_S19_B01 transporting from Armour_Stock to Armour_Layer_18 complete
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
Nothing to move
T=1559441391.07 Stop condition is satisfied, Levvel_Placement_S18_B02 transporting f

### 5. Postprocessing

In [51]:
vessels = [gantry_crane, 
           installation_crane, 
           transport_barge_01, 
           transport_barge_02, 
           hopper]
activities = ['loading', 'unloading', 'sailing full', 'sailing empty', 'processing']
colors = {0:'rgb(55,126,184)', 1:'rgb(255,150,0)', 2:'rgb(98, 192, 122)', 3:'rgb(98, 141, 122)', 4:'rgb(110, 141, 122)'}

plot.vessel_planning(vessels, activities, colors)

In [52]:
plot.vessel_kml(my_env, vessels, stepsize = 600)

In [53]:
# open the file
if platform.system():
    !start ./vessel_movements.kml
else:
    !start explorer ./vessel_movements.kml